# Convolutional Neural Network Architecture

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Flatten,  Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import Precision, Recall

## Loading the Data
To ensure efficient memory management, we utilize generators to load images in bathces.

In [3]:
data_generator = ImageDataGenerator()
train_generator = data_generator.flow_from_directory(
    directory="./images/train",
    target_size=(128, 128),
    color_mode="grayscale",
    batch_size=64,
    class_mode="binary")

Found 2392 images belonging to 2 classes.


## Convolutional Neural Network (CNN) Architecture
Our CNN is structured to efficiently process and classify grayscale images into binary categories. The architecture includes:

- `Input Layer:` Accepts images of size 128x128 pixels, with 1 channel (grayscale).
- `Convolutional Layers:` Utilize 64 filters with a kernel size of 5x5 to extract features. The 'same' padding ensures the output feature map has the same dimensions as the input.
- `Max Pooling:` Reduces the spatial dimensions, which decreases the computational load and helps in extracting dominant features while reducing overfitting.
- `Dropout Layers:` Randomly sets input units to 0 with a frequency of rate at each step during training time, which helps prevent overfitting. The rates are strategically set after pooling and dense layers.
- `Batch Normalization:` Stabilizes the learning process and dramatically reduces the number of training epochs required to train deep networks.
- `Flatten Layer:` Converts the 2D matrix data to a vector that can be fed into the dense neural network classifier.
- `Dense Layers:` The first dense layer acts as a fully connected layer for learning non-linear combinations of the high-level features extracted by the convolutional layers. The final layer outputs the prediction probability with a sigmoid activation function.

In [5]:
model = Sequential([
    Input(shape=(128, 128, 1), 
          batch_size=64),
    Conv2D(filters=64, 
           kernel_size=3, 
           padding="same", 
           activation="relu"),
    MaxPooling2D(pool_size=(5, 5)),
    Dropout(rate=0.25),
    BatchNormalization(),
    Flatten(),
    Dense(units=64,
          activation="relu"),
    Dropout(rate=0.25),
    Dense(units=1, 
          activation="sigmoid")])

## Model Compilation Strategy
For the compilation of our CNN, we opt for the following configurations:

- `Loss Function:` 'Binary Crossentropy' is used as it is suitable for binary classification problems.
- `Optimizer:` Adam optimizer is selected for its effectiveness in managing learning rates and for its adaptive property of adjusting them as per the requirement of the application.
- `Metrics:` 'Accuracy', 'Precision' and 'Recall' are monitored as the primary metrics to evaluate the performance of the model during training and testing phases.

In [6]:
model.compile(loss="binary_crossentropy",
              optimizer=Adam(learning_rate=0.001),
              metrics=["accuracy", Precision(), Recall()])

In [7]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (64, 128, 128, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (64, 25, 25, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (64, 25, 25, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (64, 25, 25, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (64, 40000)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (64, 64)               │     2,560,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (64, 64)               │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (64, 1)                │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,561,025 (9.77 MB)

 Trainable params: 2,560,897 (9.77 MB)

 Non-trainable params: 128 (512.00 B)

## Training Process and Callbacks Implementation
The model is trained on the augmented dataset, employing specific strategies to ensure efficient learning and to avoid overfitting:

- `Early Stopping:` Monitors the loss and stops the training when it begins to increase, indicating overfitting. It also restores the best model weights to ensure we retain the model that performs best on the validation set.
- `Reduce Learning Rate on Plateau:` Observes the loss and reduces the learning rate when it seems to plateau. This allows for finer adjustments to the weights, potentially leading to better performance with reduced learning rates.
- `Epochs:` The model is set to train for a maximum of 100 epochs.

In [9]:
model.fit(
    train_generator,
    epochs=100,
    verbose=False,
    callbacks=[
        EarlyStopping(monitor="loss", 
                      min_delta=0.001, 
                      patience=10, 
                      verbose=False, 
                      restore_best_weights=True),
        ReduceLROnPlateau(monitor="loss", 
                          factor=0.2, 
                          patience=5, 
                          verbose=False, 
                          min_lr=0.00001)])

In [10]:
model.save("cnn.keras")